# Import the packages used in this file

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import re
from collections import Counter 
import numpy as np

# Get a list of tuples that has the name of all games and their url

In [31]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

base_url = 'https://www.metacritic.com/browse/games/score/metascore/all/pc'
res = requests.get(base_url, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")

last_page = int(soup.find('li', class_='page last_page').find('a').text)

game_data = []

for page in range(0, last_page):
    url = base_url + f'?page={page}'
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    
    game_links = soup.find_all("a", class_="title")
    
    for link in game_links:
        game_name = link.text.strip()
        game_url = 'https://www.metacritic.com' + link['href'] + '/user-reviews'
        game_data.append((game_name, game_url))

# Get the numger of comment pages each game has and include them into the tuples

In [23]:
game_urls = []
for game in game_data:
    name, url = game
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    pagination = soup.find('li', class_='page last_page')
    if pagination:
        page_number = int(pagination.find('a', class_='page_num').text)
    game_urls.append((name, url, page_number))

# Scrape the comments using the list of tuples: [('name', 'url', page_number)]

In [25]:
def get_stats(game_urls):
    """
    Scrape user reviews for multiple games and store them in a list of lists.

    Input:
    game_urls (list): a list of tuples containing the name, url, and number of pages of user reviews for each game

    Output:
    list: a list of lists containing the game name and its corresponding list of user reviews
    """
    all_comments = []
    with requests.Session() as session:
        for game_name, url, num_pages in game_urls:
            comments = []
            for page_num in range(num_pages):
                page_url = f"{url}/user-reviews?page={page_num}"
                try:
                    response = session.get(page_url, headers=headers, allow_redirects=False)
                    if response.status_code == 200:
                        soup = BeautifulSoup(response.content, 'html.parser')
                        comment_tags = soup.find_all('div', {'class': 'review_body'})
                        comments += [comment.get_text().strip() for comment in comment_tags]
                    else:
                        print(f"Error: Could not retrieve page {page_num} for {game_name}.")
                except TypeError:
                    print(f"Error: {game_name} comments are NoneType and cannot be subscripted.")
                time.sleep(1)  # Add a delay between requests
            all_comments.append([game_name] + comments)
            print(f"Processed {game_name}, found {len(comments)} comments.")
    return all_comments

result = get_stats(game_urls)

# Convert the output list of lists into pandas dataframe

In [29]:
df = pd.DataFrame(columns=['Game', 'Comment'])

for inner_list in result:
    game = inner_list[0]
    comments = inner_list[1:]
    for comment in comments:
        df = df.append({'Game': game, 'Comment': comment}, ignore_index=True)

df

,Game,Comment
0,Disco Elysium: The Final Cut,"Still one of the best rpgs ever created, but n..."
1,Disco Elysium: The Final Cut,Communist propaganda garbage. Not worth playin...
2,Disco Elysium: The Final Cut,I'm reading through some of these negative rev...
3,Disco Elysium: The Final Cut,The game's strengths come from the intrigue of...
4,Disco Elysium: The Final Cut,One of the most unique and best written games ...
...,...,...
457150,Ride to Hell: Retribution,yknow what? this game is fun. story was bland....
457151,Ride to Hell: Retribution,This game is so hilariously bad its so good!! ...
457152,Ride to Hell: Retribution,What a prophetic title. Since the first announ...
457153,Ride to Hell: Retribution,The attempt to stick together some assets and ...


In [30]:
df.to_csv('Metacritic comments.csv', index=False) #Save the data for later use

# Convert comments into numeric values: total comments, #comments with keywords, %comments with keywords

In [31]:
df = pd.read_csv('Metacritic comments.csv') #Read data that saved earlier

Ncomments = df.groupby('Game').agg({'Comment': 'size'})

Ncomments

,Comment
Game,
.hack//G.U. Last Recode,25
007: NightFire,21
007: Quantum of Solace,23
0RBITALIS,6
10 Second Ninja,17
...,...
imprint-X,6
inMomentum,21
kill.switch,15


In [32]:
keywords = ['dizzy','dizziness','headache','tired','eyesore','fatigue','Dizzy','Dizziness','Tired', 'Headache','Eyesore','Fatigue']

# identify the rows that contain NaN values in the 'Comment' column
na_mask = df['Comment'].isna()

# filter the DataFrame to only include rows where the 'Comment' column contains any of the keywords and does not have any NaN values
filtered_df = df[df['Comment'].str.contains('|'.join(keywords)) & ~na_mask]

# group the filtered DataFrame by the 'Game' column and count the number of comments in each group
Ndiscomfort = filtered_df.groupby('Game').agg({'Comment': 'count'}).reset_index()

Ndiscomfort

,Game,Comment
0,A Game of Thrones: Genesis,1
1,A Hat in Time,1
2,A New Beginning - Final Cut,2
3,A Plague Tale: Innocence,2
4,A Plague Tale: Requiem,1
...,...,...
1159,Yu-Gi-Oh! Master Duel,1
1160,Zeno Clash,1
1161,Zeno Clash II,1
1162,Zombie Night Terror,2


In [33]:
merged_df = pd.merge(Ncomments, Ndiscomfort, on='Game', how='left')

merged_df = merged_df.rename(columns={'Comment_x': 'Ncomments'})

merged_df = merged_df.rename(columns={'Comment_y': 'Ndiscomfort'})

merged_df['Ndiscomfort'] = merged_df['Ndiscomfort'].fillna(0)

merged_df['Pdiscomfort'] = merged_df['Ndiscomfort']/merged_df['Ncomments']

merged_df

,Game,Ncomments,Ndiscomfort,Pdiscomfort
0,.hack//G.U. Last Recode,25,0.0,0.0
1,007: NightFire,21,0.0,0.0
2,007: Quantum of Solace,23,0.0,0.0
3,0RBITALIS,6,0.0,0.0
4,10 Second Ninja,17,0.0,0.0
...,...,...,...,...
5287,imprint-X,6,0.0,0.0
5288,inMomentum,21,0.0,0.0
5289,kill.switch,15,0.0,0.0
5290,nail'd,14,0.0,0.0


In [34]:
merged_df.to_csv('Cleaned Comments.csv', index=False) #Save the complete first set of data

# Scrape the APP ID for each games from SteamDB

In [1]:
!pip install selenium

In [4]:
s = Service("C:/Users/derek/OneDrive - Denison University/Downloads/chromedriver.exe") #Selenium setup
chrome_options = Options()
chrome_options.binary_location = "C:/Users/derek/AppData/Local/Google/Chrome SxS/Application/chrome.exe"

driver = webdriver.Chrome(service=s, options=chrome_options)
driver.get("https://steamdb.info/charts/")

game_list = []

while True:
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    for game in soup.find_all('td'):
        a_tag = game.find('a')
        if a_tag:
            game_name = a_tag.text.strip()
            app_id = a_tag['href'].split('/')[2]
            game_list.append([game_name, app_id])

    next_button = driver.find_element(By.CSS_SELECTOR, 'a#table-apps_next')
    if 'disabled' in next_button.get_attribute('class'):
        break
    
    next_button.click()

driver.quit()

[['', '730'], ['Counter-Strike: Global Offensive', '730'], ['', '570'], ['Dota 2', '570'], ['', '1172470'], ['Apex Legends', '1172470'], ['', '218'], ['Source SDK Base 2007', '218'], ['', '1085660'], ['Destiny 2', '1085660'], ['', '578080'], ['PUBG: BATTLEGROUNDS', '578080'], ['', '440'], ['Team Fortress 2', '440'], ['', '252490'], ['Rust', '252490'], ['', '1938090'], ['Call of Duty®: Modern Warfare® II | Warzone™ 2.0', '1938090'], ['', '1599340'], ['Lost Ark', '1599340'], ['', '2050650'], ['Resident Evil 4', '2050650'], ['', '271590'], ['Grand Theft Auto V', '271590'], ['', '218620'], ['PAYDAY 2', '218620'], ['', '236390'], ['War Thunder', '236390'], ['', '359550'], ["Tom Clancy's Rainbow Six Siege", '359550'], ['', '431960'], ['Wallpaper Engine', '431960'], ['', '1782210'], ['Crab Game', '1782210'], ['', '289070'], ["Sid Meier's Civilization VI", '289070'], ['', '346110'], ['ARK: Survival Evolved', '346110'], ['', '1811260'], ['EA SPORTS™ FIFA 23', '1811260'], ['', '1245620'], ['ELDE

In [10]:
Ngame_list = [game for game in game_list if '' not in game] #Remove the lists with empty strings

[['Counter-Strike: Global Offensive', '730'], ['Dota 2', '570'], ['Apex Legends', '1172470'], ['Source SDK Base 2007', '218'], ['Destiny 2', '1085660'], ['PUBG: BATTLEGROUNDS', '578080'], ['Team Fortress 2', '440'], ['Rust', '252490'], ['Call of Duty®: Modern Warfare® II | Warzone™ 2.0', '1938090'], ['Lost Ark', '1599340'], ['Resident Evil 4', '2050650'], ['Grand Theft Auto V', '271590'], ['PAYDAY 2', '218620'], ['War Thunder', '236390'], ["Tom Clancy's Rainbow Six Siege", '359550'], ['Wallpaper Engine', '431960'], ['Crab Game', '1782210'], ["Sid Meier's Civilization VI", '289070'], ['ARK: Survival Evolved', '346110'], ['EA SPORTS™ FIFA 23', '1811260'], ['ELDEN RING', '1245620'], ['Terraria', '105600'], ['Warframe', '230410'], ['Stardew Valley', '413150'], ['MIR4', '1623660'], ['The Sims™ 4', '1222670'], ['Football Manager 2023', '1904540'], ['Unturned', '304930'], ['Valheim', '892970'], ['DayZ', '221100'], ['Project Zomboid', '108600'], ['Hogwarts Legacy', '990080'], ['VRChat', '43810

# Convert the combination of game names and ids into Steam urls, and scrape the system requirement and release date for each game from Steam

In [ ]:
#Convert to the games to urls
url_list = [f"https://store.steampowered.com/app/{app_id}/{game_name.replace(' ', '_')}/" for game_name, app_id in Ngame_list] 

In [ ]:
exclude_list = ['https://store.steampowered.com/app/525480/.hack//G.U._Last_Recode/',
                'https://store.steampowered.com/app/841860/#Have_A_Sticker/',
                'https://store.steampowered.com/app/448350/Case_#8/']

new_list = [x for x in url_list if x not in exclude_list] #Exclude the urls that does not work

In [ ]:
system_requirements = []

for url in new_list:
    response = requests.get(url)
    if response.url == 'https://store.steampowered.com/':
        continue #ignore the ones that return back to Steam home page
        
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('div', class_='apphub_AppName').get_text().strip()
    reqs = ""

    req_div = soup.find('div', class_='game_area_sys_req sysreq_content active')
    if req_div is not None:
        try:
            reqs = req_div.find('div', class_='game_area_sys_req_rightCol').find('ul', class_='bb_ul').get_text().strip()
        except AttributeError:
            ul = req_div.find('ul', class_='bb_ul')
            if ul is not None:
                lis = ul.find_all('li')
                for li in lis:
                    reqs += li.text.strip()
            else:
                req_ps = req_div.find_all('p')
                if len(req_ps) > 1:
                    reqs = req_ps[1].text
    system_requirements.append([title, reqs]) #obtain games with their system requirements

In [ ]:
date = []
for url in new_list:
    response = requests.get(url)
    if response.url == 'https://store.steampowered.com/':
        continue #ignore the ones that return back to Steam home page
        
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('div', class_='apphub_AppName').get_text().strip()
    try:
        release_date = soup.find('div', class_='date').get_text().strip()
    except AttributeError:
        release_date = ''
    
    date.append([title, release_date]) #obtain games with their release date

In [393]:
df = pd.DataFrame(system_requirements, columns=['game_name', 'sys_req'])
datedf = pd.DataFrame(date, columns=['game_name', 'release_date'])

newdf = pd.merge(df, datedf, on='game_name')
newdf = newdf.drop_duplicates(subset=['game_name'])

newdf

,game_name,sys_req,release_date
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,"Aug 21, 2012"
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,"Jul 9, 2013"
2,Apex Legends™,Requires a 64-bit processor and operating syst...,"Nov 4, 2020"
3,Destiny 2,Requires a 64-bit processor and operating syst...,"Oct 1, 2019"
4,PUBG: BATTLEGROUNDS,Requires a 64-bit processor and operating syst...,"Dec 21, 2017"
...,...,...,...
5678,Tribloos 2,OS: Windows 7Processor: 2ghz Dual-CoreMemory: ...,"Jan 12, 2013"
5679,Might & Magic Heroes Online,Memory: 4 GB RAMGraphics: Hardware Accelerated...,"Nov 24, 2015"
5680,2 Ninjas 1 Cup,OS: Windows XPProcessor: Intel Pentium IV 2.8 ...,"Feb 3, 2017"
5681,ThrounnelVR,"OS: Windows 7, 8, 10Processor: 2.0Ghz dual-cor...","Sep 15, 2016"


# Create a new column that transform the format of game names in both datasets and merge them together

In [38]:
def transform_game_name(name):
    """
    Converts a game name to a standardized format.

    Input:
    name (str): The name of the game.

    Output:
    str: The standardized game name.
    """
    return name.lower().replace(' ', '').replace(',', '').replace(':', '').replace('-', '').replace('.', '').replace("'", '')

def clean_name(name):
    """
    Removes all non-alphanumeric characters from a string.

    Input:
    name (str): The string to clean.

    Output:
    str: The cleaned string.
    """
    name = re.sub(r'[^\w\s]', '', name)
    return name


newdf['name'] = newdf['game_name'].apply(lambda x: transform_game_name(x))
newdf['name'] = newdf['name'].apply(clean_name)

comments = pd.read_csv('Cleaned comments.csv')
comments['name'] = comments['Game'].apply(lambda x: transform_game_name(x))
comments['name'] = comments['name'].apply(clean_name)


In [395]:
merged_df = pd.merge(newdf, comments, on='name')

merged_df = merged_df.drop('Game', axis=1)

merged_df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,"Aug 21, 2012",counterstrikeglobaloffensive,1148,22.0,0.019164
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,"Jul 9, 2013",dota2,3829,17.0,0.004440
2,Apex Legends™,Requires a 64-bit processor and operating syst...,"Nov 4, 2020",apexlegends,550,3.0,0.005455
3,Destiny 2,Requires a 64-bit processor and operating syst...,"Oct 1, 2019",destiny2,559,4.0,0.007156
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,"Oct 10, 2007",teamfortress2,1635,16.0,0.009786
...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,"Jul 19, 2011",il2sturmovikcliffsofdover,76,0.0,0.000000
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,,dearesther,265,2.0,0.007547
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,"Apr 5, 2010",plainsight,18,1.0,0.055556
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,"Nov 4, 2009",shatteredhorizon,41,0.0,0.000000


In [ ]:
newdf.to_csv('raw_sys.csv', index=False) #Save data for later use
comments.to_csv('Cleaned Comments.csv', index=False) #Save data for later use
merged_df.to_csv('Finaldf.csv', index=False) #Save data for later use

# Seperate the raw system requirement and extract each as a new column

In [204]:
df = pd.read_csv('Finaldf.csv')

keys = ['OS:', 'Processor:', 'Memory:', 'Graphics:', 'DirectX:', 'Storage:', 'Network:']
req = list(df['sys_req'])
parsed = []
for e, i in enumerate(req):
    d = str(i)
    for z in keys:
        d = d.replace(z, '||'+z+'||')
    fields = d.split('||')

    these_keys = []
    these_vals = [] 
    for y in range(1, len(fields), 1):
        if y % 2 == 1:
            these_keys.append(fields[y])
        else:
            these_vals.append(fields[y])
    these_keys = [o.replace(':', '') for o in these_keys]

    if len(these_keys) == len(these_vals):
        if len(these_keys) == 0:
            result = {'no fields':e}
        else:
            result = dict(list(zip(these_keys, these_vals)))
    else:
        result = {'error': e}
    parsed.append(result)

reqdf = pd.DataFrame(parsed)
df = pd.concat([df, reqdf], axis=1)
df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,Graphics,DirectX,Storage,Network,no fields
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,Video card must be 256 MB or more and should ...,Version 9.0c,15 GB available space,NaN,NaN
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,"NVIDIA GeForce 8600/9600GT, ATI/AMD Radeon HD...",Version 11,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,"AMD Radeon™ R9 290, NVIDIA GeForce® GTX 970",Version 11,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,NVIDIA® GeForce® GTX 970 4GB or GTX 1060 6GB ...,NaN,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,NaN,Version 9.0c,15 GB available space,Broadband Internet connection,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,"DirectX® 10 compliant, 1GB Video Card (See su...",NaN,NaN,NaN,NaN
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,DirectX 9 compliant video card with Shader mod...,NaN,NaN,NaN,NaN
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,Shader model 2 supported graphics cardDirectX...,NaN,NaN,NaN,NaN
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,512MB NVIDIA GeForce GTX 260 / ATI Radeon HD ...,NaN,NaN,NaN,NaN


In [205]:
all_os = []
for i in parsed:
    try:
        data = i['Processor']
        all_os.append(data)
    except:
        pass
Counter(all_os).most_common(100) #Check the unique values and the number of their presence

[(' 2 GHz', 6),
 (' Intel Core i5', 5),
 (' Dual Core Processor', 5),
 (' 2.0 Ghz', 4),
 (' Intel Core i7', 4),
 (' AMD/INTEL DUAL-CORE 2.5 GHZ', 4),
 (' Dual Core CPU', 4),
 (' Quad Core CPU', 4),
 (' Intel® Core™ i5-4570 3.20GHz', 3),
 (' 3GHz Quad Core', 3),
 (' AMD Ryzen™ 3 1200 / Intel® Core™ i5-6400', 3),
 (' Intel® Pentium® IV 2.4 GHz or AMD 3500+', 3),
 (' Quad-core Intel or AMD CPU ', 3),
 (' 3.2 GHz Quad Core Processor', 3),
 (' Core i5 or equivalent', 3),
 (' Intel Core 2 Duo E6400 / AMD 64 X2 4200+ ', 3),
 (' Intel Core i7-4790, AMD FX-8350', 3),
 (' Intel Core i5 4690 or AMD FX 8320', 3),
 (' Dual-core CPU with SSE3 (Intel® Pentium® D 3GHz / AMD Athlon™ 64 X2 4200) or better',
  3),
 (' 1.7 GHz Dual Core or Greater', 3),
 (' AMD/INTEL DUAL-CORE 2.4 GHZ', 3),
 (' AMD Ryzen 5 2600 / Intel Core i5-8600K', 3),
 (' Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 8750 processor or better', 2),
 (' Dual core from Intel or AMD at 2.8 GHz', 2),
 (' 2 Ghz', 2),
 (' Quad core CPU 3.0 GHz'

# Memory -> numeric

In [206]:
df['memory_num'] = df['Memory'].str.extract('(\d+)\s*(?:GB|MB)\s*', flags=re.IGNORECASE) #Extract the nominal numbers
df['memory_num'] = pd.to_numeric(df['memory_num'], errors='coerce')
df.loc[df['memory_num'] > 100, 'memory_num'] /= 1024 #Unify units
df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,Graphics,DirectX,Storage,Network,no fields,memory_num
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,Video card must be 256 MB or more and should ...,Version 9.0c,15 GB available space,NaN,NaN,2.0
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,"NVIDIA GeForce 8600/9600GT, ATI/AMD Radeon HD...",Version 11,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,"AMD Radeon™ R9 290, NVIDIA GeForce® GTX 970",Version 11,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,NVIDIA® GeForce® GTX 970 4GB or GTX 1060 6GB ...,NaN,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,NaN,Version 9.0c,15 GB available space,Broadband Internet connection,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,"DirectX® 10 compliant, 1GB Video Card (See su...",NaN,NaN,NaN,NaN,4.0
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,DirectX 9 compliant video card with Shader mod...,NaN,NaN,NaN,NaN,1.0
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,Shader model 2 supported graphics cardDirectX...,NaN,NaN,NaN,NaN,1.0
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,512MB NVIDIA GeForce GTX 260 / ATI Radeon HD ...,NaN,NaN,NaN,NaN,2.0


# OS -> numeric

In [208]:
df['OS_num'] = df['OS'].apply(lambda x: 0 
                              if pd.isna(x) 
                                  else (6 if '11' in x 
                                        else (5 if '10' in x 
                                              else (3 if '8' in x 
                                                    else (2 if '7' in x 
                                                          else (1 if 'XP' in x else 0))))))    #Assign values to OS
df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,Graphics,DirectX,Storage,Network,no fields,memory_num,OS_num
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,Video card must be 256 MB or more and should ...,Version 9.0c,15 GB available space,NaN,NaN,2.0,2
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,"NVIDIA GeForce 8600/9600GT, ATI/AMD Radeon HD...",Version 11,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,"AMD Radeon™ R9 290, NVIDIA GeForce® GTX 970",Version 11,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,2
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,NVIDIA® GeForce® GTX 970 4GB or GTX 1060 6GB ...,NaN,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,5
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,NaN,Version 9.0c,15 GB available space,Broadband Internet connection,NaN,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,"DirectX® 10 compliant, 1GB Video Card (See su...",NaN,NaN,NaN,NaN,4.0,2
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,DirectX 9 compliant video card with Shader mod...,NaN,NaN,NaN,NaN,1.0,1
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,Shader model 2 supported graphics cardDirectX...,NaN,NaN,NaN,NaN,1.0,1
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,512MB NVIDIA GeForce GTX 260 / ATI Radeon HD ...,NaN,NaN,NaN,NaN,2.0,2


In [209]:
for i, row in df.iterrows(): #Accumulate the OS value if it requires 64 bit
    if '64bit' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64-bit' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64 bit' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64 BIT' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64 Bit' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64-Bit' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64-bits' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64-BIT' in str(row['OS']):
        df.at[i, 'OS_num'] += 1
    elif '64Bit' in str(row['OS']):
        df.at[i, 'OS_num'] += 1

df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,Graphics,DirectX,Storage,Network,no fields,memory_num,OS_num
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,Video card must be 256 MB or more and should ...,Version 9.0c,15 GB available space,NaN,NaN,2.0,2
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,"NVIDIA GeForce 8600/9600GT, ATI/AMD Radeon HD...",Version 11,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,"AMD Radeon™ R9 290, NVIDIA GeForce® GTX 970",Version 11,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,3
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,NVIDIA® GeForce® GTX 970 4GB or GTX 1060 6GB ...,NaN,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,6
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,NaN,Version 9.0c,15 GB available space,Broadband Internet connection,NaN,1.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,"DirectX® 10 compliant, 1GB Video Card (See su...",NaN,NaN,NaN,NaN,4.0,2
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,DirectX 9 compliant video card with Shader mod...,NaN,NaN,NaN,NaN,1.0,1
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,Shader model 2 supported graphics cardDirectX...,NaN,NaN,NaN,NaN,1.0,1
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,512MB NVIDIA GeForce GTX 260 / ATI Radeon HD ...,NaN,NaN,NaN,NaN,2.0,2


# Storage -> numeric

In [210]:
for index, row in df.iterrows():
    storage_value = re.findall(r'^\s(\d{1,4})', str(row['Storage']))
    if storage_value:
        df.at[index, 'storage_num'] = int(storage_value[0])

for i in range(len(df['storage_num'])):
    if df['storage_num'][i] > 180:
        df['storage_num'][i] = round(df['storage_num'][i] / 1024, 3) #Unify units

df

<ipython-input-210-aa308a5553a7>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['storage_num'][i] = round(df['storage_num'][i] / 1024, 3)


,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,...,DirectX,Storage,Network,no fields,memory_num,OS_num,storage_num,graphics_simp,graphics_num,processor_simp
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,...,Version 9.0c,15 GB available space,NaN,NaN,2.0,2,15,NaN,5.0,DUO
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,...,Version 11,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2,60,GEFORCE 8600,5.2,DUAL
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,...,Version 11,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,3,56,GTX 970,6.9,NaN
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,...,NaN,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,6,105,GTX 970,6.9,I5-2400
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,...,Version 9.0c,15 GB available space,Broadband Internet connection,NaN,1.0,3,15,NaN,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,...,NaN,NaN,NaN,NaN,4.0,2,0,NaN,5.0,I5-2
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,...,NaN,NaN,NaN,NaN,1.0,1,0,NVIDIA 8800,5.2,QUAD
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,...,NaN,NaN,NaN,NaN,1.0,1,0,NaN,5.0,NaN
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,...,NaN,NaN,NaN,NaN,2.0,2,0,GTX 260,5.1,QUAD


# Graphics -> numeric (https://gpu.userbenchmark.com/)

In [224]:
pattern1 = r'(GTX\s\d{3,4}\s?(?:Ti)?)'
pattern2 = r'(GeForce\s\d{3,4}\s?(?:Ti)?)'
pattern3 = r'(RTX \d{3,4}\s?(?:Ti)?)'
pattern4 = r'(GT \d{3,4}\s?(?:Ti)?)'
pattern5 = r'(NVIDIA \d{3,4}\s?(?:Ti)?)'
pattern6 = r'(GTX\d{3,4}\s?(?:Ti)?)'

match1 = df['Graphics'].str.extract(pattern1, flags=re.IGNORECASE, expand=False)
match2 = df['Graphics'].str.extract(pattern2, flags=re.IGNORECASE, expand=False)
match3 = df['Graphics'].str.extract(pattern3, flags=re.IGNORECASE, expand=False)
match4 = df['Graphics'].str.extract(pattern4, flags=re.IGNORECASE, expand=False)
match5 = df['Graphics'].str.extract(pattern5, flags=re.IGNORECASE, expand=False)
match6 = df['Graphics'].str.extract(pattern6, flags=re.IGNORECASE, expand=False)

conditions = [match1.notnull(), match2.notnull(), match3.notnull(), match4.notnull(), match5.notnull(), match6.notnull()]
choices = [match1, match2, match3, match4, match5, match6]
df['graphics_simp'] = np.select(conditions, choices, default=np.nan)

df['graphics_simp'] = df['graphics_simp'].str.upper().str.replace(r'GTX(\d+)', r'GTX \1')

df

<ipython-input-224-603ccd1176cd>:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df['graphics_simp'] = df['graphics_simp'].str.upper().str.replace(r'GTX(\d+)', r'GTX \1')


,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,...,Storage,Network,no fields,memory_num,OS_num,storage_num,graphics_simp,graphics_num,processor_simp,processor_num
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,...,15 GB available space,NaN,NaN,2.0,2,15,NaN,5.0,DUO,36.6
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,...,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2,60,GEFORCE 8600,5.2,DUAL,36.6
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,...,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,3,56,GTX 970,6.9,NaN,27.7
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,...,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,6,105,GTX 970,6.9,I5-2400,60.6
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,...,15 GB available space,Broadband Internet connection,NaN,1.0,3,15,NaN,5.0,NaN,27.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,...,NaN,NaN,NaN,4.0,2,0,NaN,5.0,I5-2,42.0
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,...,NaN,NaN,NaN,1.0,1,0,NVIDIA 8800,5.2,QUAD,45.5
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,...,NaN,NaN,NaN,1.0,1,0,NaN,5.0,NaN,27.7
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,...,NaN,NaN,NaN,2.0,2,0,GTX 260,5.1,QUAD,45.5


In [227]:
mapping = {'RTX 3070': 148, 'RTX 2080': 124, 'RTX 3060': 100, 'RTX 2070': 104, 'RTX 2060': 89.8, 'NVIDIA 9800': 7.51, 
           'NVIDIA 970': 49.1, 'NVIDIA 9400': 1.08, 'NVIDIA 8800': 7.11, 'NVIDIA 8600': 5.41, 'NVIDIA 8000': 5.41, 
           'NVIDIA 7900': 1.9, 'NVIDIA 7800': 1.31, 'NVIDIA 770': 34.1, 'NVIDIA 7600': 0.87, 'NVIDIA 6800': 0.16, 'NVIDIA 6600': 0.4, 
           'NVIDIA 500': 10.1, 'NVIDIA 460': 14.2, 'NVIDIA 320': 3.45, 'NVIDIA 260': 7.31, 'NVIDIA 1050 TI': 29.3, 'GTX 980': 59.3, 
           'GTX 970': 49.1, 'GTX 960': 32.9, 'GTX 950': 25.6, 'GTX 900': 33.4, 'GTX 780': 44.9, 'GTX 770': 34.1, 'GTX 760': 26, 
           'GTX 750': 14.7, 'GTX 740': 7.43, 'GTX 680': 31.6, 'GTX 670': 27.7, 'GTX 660': 19.9, 'GTX 650': 8.18, 'GTX 630': 3.62, 
           'GTX 600': 6.46, 'GTX 580': 22.8, 'GTX 570': 19.9, 'GTX 560': 13.8, 'GTX 555': 11.5, 'GTX 500': 7.4, 
           'GTX 480': 19.6, 'GTX 470': 16.2, 'GTX 460': 12.1, 'GTX 450': 7.38, 'GTX 400': 6.41, 'GTX 280': 8.47, 'GTX 275': 9.02, 
           'GTX 260': 7.31, 'GTX 2070': 104, 'GTX 1660': 68.3, 'GTX 1650': 42.8, 'GTX 1080': 102, 'GTX 1070': 79.7, 'GTX 1060': 56.1,
           'GTX 1050': 25.7, 'GTX 1000': 25.7, 'GT 760': 10.7, 'GT 750': 7.68, 'GT 730': 4.87, 'GT 650': 6.5, 'GT 640': 5.13, 
           'GT 610': 1.84, 'GT 512': 7.11, 'GT 450': 7.38, 'GT 430': 3.06, 'GT 240': 4.05, 'GT 220': 2.33, 'GT 1060': 46.7, 
           'GT 1030': 13.6, 'GEFORCE 9800': 6.48, 'GEFORCE 970': 30.7, 'GEFORCE 9600': 4.13, 'GEFORCE 960': 16.4, 'GEFORCE 9500': 2.04, 
           'GEFORCE 9400': 1.08, 'GEFORCE 9000': 0.55, 'GEFORCE 900': 0.96, 'GEFORCE 8800': 5.65, 
           'GEFORCE 8600': 2.25, 'GEFORCE 8000': 8.48, 'GEFORCE 7900': 1.9, 'GEFORCE 7800': 1.31, 'GEFORCE 780': 19.9, 
           'GEFORCE 770': 16, 'GEFORCE 7600': 0.87, 'GEFORCE 760': 10.7, 'GEFORCE 750': 7.68, 'GEFORCE 710': 3.48, 
           'GEFORCE 700': 6.43, 'GEFORCE 6800': 1.04, 'GEFORCE 6600': 0.4, 'GEFORCE 660': 7.26, 'GEFORCE 650': 6.5, 
           'GEFORCE 640': 5.13, 'GEFORCE 6100': 0.05, 'GEFORCE 610': 1.84, 'GEFORCE 600': 1.84, 'GEFORCE 560': 7.32, 
           'GEFORCE 500': 1.75, 'GEFORCE 480': 9.19, 'GEFORCE 470': 16.2, 'GEFORCE 460': 6.41, 'GEFORCE 450': 7.38, 
           'GEFORCE 400': 1.96, 'GEFORCE 320': 1.88, 'GEFORCE 260': 4.8, 'GEFORCE 240': 2.31, 'GEFORCE 2060': 89.8, 
           'GEFORCE 1080': 102, 'GEFORCE 1060': 56.1, 'GEFORCE 1050': 25.7, 'GTX 980 TI': 79, 'GTX 980TI': 79, 
           'GTX 780 TI': 52.9, 'GTX 750 TI': 17.2, 'GTX 750TI': 17.2, 'GTX 660 TI': 25.9, 'GTX 660TI': 25.9, 'GTX 650TI': 14.1, 
           'GTX 650 TI': 14.1, 'GTX 560TI': 16.3, 'GTX 560 TI': 16.3, 'GTX 550TI': 8.61, 'GTX 550 TI': 8.61, 'GTX 1660TI': 76, 
           'GTX 1660 TI': 76, 'GTX 1080 TI': 124, 'GTX 1070 TI': 93.7, 'GTX 1050 TI': 29.3, 'GTX 1050TI': 29.3, 
           'GEFORCE 750 TI': 17.2, 'GEFORCE 660 TI': 25.9, 'GEFORCE 560 TI': 16.3, 'GEFORCE 1080 TI': 124, 'GEFORCE 1050 TI': 29.3}

df['graphics_num'] = df['graphics_simp'].map(mapping).fillna(2) #Mapping with benmarks

df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,...,Storage,Network,no fields,memory_num,OS_num,storage_num,graphics_simp,graphics_num,processor_simp,processor_num
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,...,15 GB available space,NaN,NaN,2.0,2,15,NaN,2.00,DUO,36.6
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,...,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2,60,GEFORCE 8600,2.25,DUAL,36.6
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,...,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,3,56,GTX 970,49.10,NaN,27.7
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,...,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,6,105,GTX 970,2.00,I5-2400,60.6
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,...,15 GB available space,Broadband Internet connection,NaN,1.0,3,15,NaN,2.00,NaN,27.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,...,NaN,NaN,NaN,4.0,2,0,NaN,2.00,I5-2,42.0
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,...,NaN,NaN,NaN,1.0,1,0,NVIDIA 8800,7.11,QUAD,45.5
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,...,NaN,NaN,NaN,1.0,1,0,NaN,2.00,NaN,27.7
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,...,NaN,NaN,NaN,2.0,2,0,GTX 260,2.00,QUAD,45.5


# Processor -> numeric (https://cpu.userbenchmark.com/)

In [213]:
pattern1 = r'(i\d[-\s]?[0-9]+[CTKFS]?\b)'
pattern2 = r'(duo|dual|quad)'
pattern3 = r'(i\d\s*[- ]\s*\d+[KS]?\b)'


match1 = df['Processor'].str.extract(pattern1, flags=re.IGNORECASE, expand=False)
match2 = df['Processor'].str.extract(pattern2, flags=re.IGNORECASE, expand=False)
match3 = df['Processor'].str.extract(pattern3, flags=re.IGNORECASE, expand=False)

conditions = [match1.notnull(), match2.notnull(), match3.notnull()]
choices = [match1, match2, match3]
df['processor_simp'] = np.select(conditions, choices, default=np.nan)
df['processor_simp'] = df['processor_simp'].str.upper().str.replace(' ', '-')
df['processor_simp'] = df['processor_simp'].str.replace('--', '-')

df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,...,DirectX,Storage,Network,no fields,memory_num,OS_num,storage_num,graphics_simp,graphics_num,processor_simp
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,...,Version 9.0c,15 GB available space,NaN,NaN,2.0,2,15,NaN,5.0,DUO
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,...,Version 11,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2,60,GEFORCE 8600,5.2,DUAL
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,...,Version 11,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,3,56,GTX 970,6.9,NaN
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,...,NaN,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,6,105,GTX 970,6.9,I5-2400
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,...,Version 9.0c,15 GB available space,Broadband Internet connection,NaN,1.0,3,15,NaN,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,...,NaN,NaN,NaN,NaN,4.0,2,0,NaN,5.0,I5-2
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,...,NaN,NaN,NaN,NaN,1.0,1,0,NVIDIA 8800,5.2,QUAD
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,...,NaN,NaN,NaN,NaN,1.0,1,0,NaN,5.0,NaN
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,...,NaN,NaN,NaN,NaN,2.0,2,0,GTX 260,5.1,QUAD


In [215]:
mapping = {'I9-10900K': 99.6, 'I7-9700K': 96.9, 'I7-9700': 92, 'I7-950': 65.9, 'I7-920': 61.6, 'I7-8700K': 88.4, 
           'I7-8700': 87, 'I7-8600K': 90.7, 'I7-8600': 88.2, 'I7-7700K': 84, 'I7-7700': 76.2, 'I7-6700K': 80.8, 
           'I7-6700': 72.3, 'I7-5930K': 79.8, 'I7-5820K': 76.8, 'I7-5775C': 84.9, 'I7-4790K': 78.9, 'I7-4790': 71.4, 
           'I7-4770S': 71.2, 'I7-4770K': 73.6, 'I7-4770': 70.5, 'I7-3970': 77.9, 'I7-3930K': 74.4, 'I7-3820': 69.5, 
           'I7--3770K': 72.4, 'I7-3770K': 72.4, 'I7-3770': 69, 'I7-3700': 60.8, 'I7-3': 60.8, 'I7-2700K': 73, 'I7-2600K': 70.6, 
           'I7-2600': 67.1, 'I7-2': 67.1, 'I7-11700': 98.1, 'I7-10700K': 96.5, 'I5-9600K': 93.2, 'I5-9600': 92.3, 'I5-9500F': 87, 
           'I5-9500': 87, 'I5-9400F': 84.8, 'I5-9400': 86.5, 'I5-8600K': 90.7, 'I5-8600': 88.2, 'I5-8500': 82.1, 'I5-8400': 81.8, 
           'I5-7600K': 75.8, 'I5-7600': 72, 'I5-760': 63.6, 'I5-7500': 66.8, 'I5-750': 61.5, 'I5-7400S': 65.3, 'I5-7400': 63.5, 
           'I5--6600K': 72.4, 'I5-6600K': 72.4, 'I5-6600': 67.6, 'I5-6500': 63.6, 'I5-650': 46.3, 'I5-6400': 60.8, 'I5-5675C': 81.8,
           'I5-4690K': 69.6, 'I5-4690': 67.1, 'I5-4670K': 68, 'I5-4670': 66.5, 'I5-4590': 63.5, 'I5-4570T': 57.7, 'I5-4570': 62.4, 
           'I5-4460': 62.1, 'I5-4440': 61.3, 'I5-4430': 65.2, 'I5-4000': 65.2, 'I5-3770': 49, 'I5-3570K': 67.6, 'I5-3570': 65.1, 
           'I5-350K': 60.8, 'I5-3470': 63.1, 'I5-3300': 60.8, 'I5-3000': 57.6, 'I5-3': 57.6, 'I5-2520': 47.6, 
           'I5-2500K': 55.2, 'I5-2500': 62.9, 'I5-2400S': 60.4, 'I5-2400': 60.6, 'I5-2320': 63.9, 
           'I5-2300': 64, 'I5-2100': 41.9, 'I5-2': 42, 'I5-11600K': 99.2, 'I5-11400': 94.2, 
           'I5-10505': 70.2, 'I3-8350K': 81.7, 'I3-8100': 70.6, 'I3-6300': 66.7, 'I3-6100T': 57.6, 
           'I3-6100': 55.4, 'I3-560': 51.6, 'I3-550': 47, 'I3-530': 45.6, 'I3-4340': 64.5, 
           'I3-4330': 61.9, 'I3-4170': 56.9, 'I3-4160': 55.1, 'I3-4130': 52.3, 'I3-3240': 55.1, 
           'I3-3': 51.4, 'I3-2120': 50.7, 'I3-2100T': 52.6, 'I3-2100': 50.9, 'I3-2': 50.9, 
           'QUAD': 45.5, 'DUO': 36.6, 'DUAL': 36.6}

df['processor_num'] = df['processor_simp'].map(mapping).fillna(27.7) #Mapping with benchmarks

df

,game_name,sys_req,release_date,name,Ncomments,Ndiscomfort,Pdiscomfort,OS,Processor,Memory,...,Storage,Network,no fields,memory_num,OS_num,storage_num,graphics_simp,graphics_num,processor_simp,processor_num
0,Counter-Strike: Global Offensive,OS: Windows® 7/Vista/XPProcessor: Intel® Core™...,21-Aug-12,counterstrikeglobaloffensive,1148,22,0.019164,Windows® 7/Vista/XP,Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 87...,2 GB RAM,...,15 GB available space,NaN,NaN,2.0,2,15,NaN,5.0,DUO,36.6
1,Dota 2,OS: Windows 7 or newerProcessor: Dual core fro...,9-Jul-13,dota2,3829,17,0.004440,Windows 7 or newer,Dual core from Intel or AMD at 2.8 GHz,4 GB RAM,...,60 GB available spaceSound Card: DirectX Comp...,Broadband Internet connection,NaN,4.0,2,60,GEFORCE 8600,5.2,DUAL,36.6
2,Apex Legends™,Requires a 64-bit processor and operating syst...,4-Nov-20,apexlegends,550,3,0.005455,64-bit Windows 7,Ryzen 5 CPU or Equivalent,8 GB RAM,...,56 GB available spaceAdditional Notes: ~3.8GB...,Broadband Internet connection,NaN,8.0,3,56,GTX 970,6.9,NaN,27.7
3,Destiny 2,Requires a 64-bit processor and operating syst...,1-Oct-19,destiny2,559,4,0.007156,System Windows® 7 / Windows® 8.1 / Windows® 1...,Processor Intel® Core™ i5 2400 3.4 GHz or i5 ...,8 GB RAM,...,105 GB available spaceAdditional Notes:,Broadband Internet connection,NaN,8.0,6,105,GTX 970,6.9,I5-2400,60.6
4,Team Fortress 2,OS: Windows® 7 (32/64-bit)Processor: Pentium 4...,10-Oct-07,teamfortress2,1635,16,0.009786,Windows® 7 (32/64-bit),"Pentium 4 processor (3.0GHz, or better)",1 GB RAM,...,15 GB available space,Broadband Internet connection,NaN,1.0,3,15,NaN,5.0,NaN,27.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,IL-2 Sturmovik: Cliffs of Dover,OS: Windows® 7 / Vista SP2 / Windows XP SP3 Pr...,19-Jul-11,il2sturmovikcliffsofdover,76,0,0.000000,Windows® 7 / Vista SP2 / Windows XP SP3,Intel Core i5 2.66GHz or AMD Phenom II X4 2.6...,4GB,...,NaN,NaN,NaN,4.0,2,0,NaN,5.0,I5-2,42.0
1424,Dear Esther,OS:Microsoft Windows XP / Vista / Vista64 Proc...,NaN,dearesther,265,2,0.007547,Microsoft Windows XP / Vista / Vista64,Quad core 2.4GHz or higher,1GB XP / 2GB Vista,...,NaN,NaN,NaN,1.0,1,0,NVIDIA 8800,5.2,QUAD,45.5
1425,Plain Sight,OS: Windows XPProcessor: 2 ghz or betterMemory...,5-Apr-10,plainsight,18,1,0.055556,Windows XP,2 ghz or better,1 GB RAM,...,NaN,NaN,NaN,1.0,1,0,NaN,5.0,NaN,27.7
1426,Shattered Horizon,OS: Windows Vista/7 (Does not support Windows ...,4-Nov-09,shatteredhorizon,41,0,0.000000,Windows Vista/7 (Does not support Windows XP),Intel Core 2 Quad Q6600 / AMD Phenom II X4 940,2GB,...,NaN,NaN,NaN,2.0,2,0,GTX 260,5.1,QUAD,45.5


In [228]:
df.to_csv('Finaldf.csv', index=False) #Save the complete data (without manual fill-in)